In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [3]:
from collections import namedtuple
WordPos = namedtuple('WordPos', 'word,section_id,para_id,sentence_id,word_id')

In [4]:
from scripts.analyse_pwc_entity_results import *

In [5]:
def add_id(row) :
    section_id, para_id, sentence_id = row['section_id'], row['para_id'], row['sentence_id']
    augmented_words = []
    words = row['words']
    for i, w in enumerate(words) :
        augmented_words.append(WordPos(word=w,section_id=section_id,para_id=para_id,sentence_id=sentence_id,word_id=i))
    return augmented_words

In [6]:
pwc_df_sentences_v3 = pd.read_json('../data/pwc_s2_cleaned_text_v3_sentences.jsonl', lines=True)

In [7]:
pwc_df_sentences_v2 = load_pwc_sentence_predictions('../data/pwc_s2_cleaned_text_v2_sentences.jsonl',
                                                   '../outputs/pwc_s2_cleaned_text_v2_sentences_predictions.jsonl')

314630it [00:57, 5454.58it/s]


314630 314630


100%|██████████| 314630/314630 [00:14<00:00, 21670.70it/s]


In [9]:
pwc_df_sentences_v3['words'] = pwc_df_sentences_v3['sentence'].apply(lambda x : x.split())
pwc_df_sentences_v3['aug_words'] = pwc_df_sentences_v3.apply(add_id, axis=1)

In [10]:
pwc_df_sentences_v2['aug_words'] = pwc_df_sentences_v2.progress_apply(add_id, axis=1)

100%|██████████| 314630/314630 [00:21<00:00, 14800.38it/s]


In [26]:
words_s2 = pwc_df_sentences_v2.rename(columns={'aug_words' : 'x'}).groupby('doc_id')['x'].agg(lambda x : [w for y in x for w in y])
words_s3 = pwc_df_sentences_v3.rename(columns={'aug_words' : 'y'}).groupby('doc_id')['y'].agg(lambda x : [w for y in x for w in y])

In [27]:
concatenated = pd.concat([words_s2, words_s3], axis=1)

In [97]:
same_sent = lambda x, y : (x['section_id'] == y['section_id']) & (x['para_id'] == y['para_id']) & (x['sentence_id'] == y['sentence_id'])
get_pos = lambda x : (x.section_id, x.para_id, x.sentence_id, x.word_id)
def match_lists(words_2, words_3) :
    if len(words_2) == len(words_3) :
        if [x.word for x in words_2] == [x.word for x in words_3] :
            return words_3
        else :
            breakpoint()
    else :
        i, j = 0, 0
        new_words = []
        curr_word = words_3[j]._asdict()
        for word in words_2 :
            if word.word == curr_word['word'] :
                new_words.append(WordPos(**curr_word)._asdict())
                j += 1
                if j >= len(words_3) : break
                curr_word = words_3[j]._asdict()
            elif curr_word['word'].startswith(word.word) :
                new_words.append(WordPos(word=word.word, 
                                         section_id=curr_word['section_id'], 
                                         para_id=curr_word['para_id'], 
                                         sentence_id=curr_word['sentence_id'],
                                         word_id=curr_word['word_id'])._asdict())
                curr_word['word'] = curr_word['word'][len(word.word):].strip()
            else :
                breakpoint()
        
        for i, word in enumerate(new_words[:-1]) :
            next_word = new_words[i+1]
            if same_sent(word, next_word) and word['word_id'] >= next_word['word_id'] :
                next_word['word_id'] = word['word_id'] + 1
                
        new_words = [WordPos(**x) for x in new_words]
        assert [x.word for x in words_2] == [x.word for x in new_words]
        for i, word in enumerate(new_words[:-1]) :
            next_word = new_words[i+1]
            assert get_pos(word) < get_pos(next_word)

        return new_words

In [98]:
concatenated.progress_apply(lambda x: match_lists(x['x'], x['y']), axis=1)



  0%|          | 0/1170 [00:00<?, ?it/s]

  1%|          | 9/1170 [00:00<00:15, 73.12it/s]

  2%|▏         | 25/1170 [00:00<00:13, 83.47it/s]

  4%|▍         | 46/1170 [00:00<00:11, 98.50it/s]

  5%|▍         | 55/1170 [00:00<00:11, 93.37it/s]

  5%|▌         | 64/1170 [00:00<00:13, 81.79it/s]

  8%|▊         | 93/1170 [00:00<00:10, 103.81it/s]

 10%|▉         | 114/1170 [00:00<00:09, 115.75it/s]

 11%|█         | 129/1170 [00:01<00:10, 98.39it/s] 

 12%|█▏        | 142/1170 [00:01<00:11, 85.77it/s]

 13%|█▎        | 153/1170 [00:01<00:11, 89.08it/s]

 14%|█▍        | 167/1170 [00:01<00:10, 91.20it/s]

 16%|█▋        | 193/1170 [00:01<00:08, 113.14it/s]

 18%|█▊        | 209/1170 [00:01<00:08, 112.27it/s]

 19%|█▉        | 223/1170 [00:01<00:08, 109.39it/s]

 20%|██        | 236/1170 [00:02<00:12, 75.15it/s] 

 22%|██▏       | 256/1170 [00:02<00:10, 86.11it/s]

 23%|██▎       | 271/1170 [00:02<00:09, 93.52it/s]

 24%|██▍       | 284/1170 [00:02<00:09, 97.65it/s]

 25%|██▌       | 296

doc_id
000f90380d768a85e2316225854fc377c079b5c4    [(Full, 0, 0, 0, 0), (-, 0, 0, 0, 1), (Resolut...
0012de6bec1f25599e4f02517637e531a71909b9    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (V, ...
007ab5528b3bd310a80d553cccad4b78dc496b02    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Bi,...
007ff2ca5f297b04636699ce4d01ca6d6f21dc77    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Att...
0095c269e7d0c990249312687fc43521019809c4    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Mod...
00b1cdc5bd77bf27f9b1ca630365eeeb456913b4    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Mas...
01125e3c68edb420b8d884ff53fb38d9fbe4f2b8    [(document, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Lar...
0116899fce00ffa4afee08b505300bb3968faf9f    [(Long, 0, 0, 0, 0), (Text, 0, 0, 0, 1), (Gene...
0171bdeb1c6e333287be655c667cfba5edb89b76    [(Aggregated, 0, 0, 0, 0), (Residual, 0, 0, 0,...
01959ef569f74c286956024866c1d107099199f7    [(VQA, 0, 0, 0, 0), (:, 0, 0, 0, 1), (Visual, ...
01c824989d24a8cae214c3156edd9d4492faa579    [(documen

In [ ]:
pwc_df_sentences_v2[(pwc_df_sentences_v2['words'].apply(lambda x : " ".join(x)) != pwc_df_sentences_v2['sentence'])]

In [ ]:
pwc_df_sentences_v2['words'] = pwc_df_sentences_v2['sentence'].apply(lambda x : x.split())

In [ ]:
def map_sentences(words_1, words_2) :
    assert [x.word for x in words_1] == [x.word for x in words_2]
    return None

In [ ]:
pwc_df_v2 = pd.read_json('../data/pwc_s2_cleaned_text_v2.jsonl', lines=True)
pwc_df_v3 = pd.read_json('../data/pwc_s2_cleaned_text_v3.jsonl', lines=True)

In [ ]:
s2 = pwc_df_sentences_v2.rename(columns={'aug_words' : 'x'}).groupby('doc_id')['x'].apply(lambda x : [w for y in x for w in y])
s3 = pwc_df_sentences_v3.rename(columns={'aug_words' : 'y'}).groupby('doc_id')['y'].apply(lambda x : [w for y in x for w in y])

In [ ]:
concat = pd.concat([s2, s3], axis=1)